# Client

Demo of a client interacting with a remote agent that can use history.

See relevant documentation about agents:

* Creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
* Streaming with agents: https://python.langchain.com/docs/modules/agents/how_to/streaming#custom-streaming-with-events
* General streaming documentation: https://python.langchain.com/docs/expression_language/streaming

You can interact with this via API directly

In [4]:
!pip install sse_starlette

In [11]:
import requests

inputs = {
    "input": {
        "input": "What are the categories in the database?",
        "chat_history": [],
    }
}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': 'The categories in the database are:\n1. misc\n2. cerrajeria\n3. piano\n4. cabello\n5. bateria\n6. french_food\n7. body_treatments\n8. 8_56_hierro\n9. home\n10. moto\n11. filmmaking\n12. auto\n13. music\n14. video\n15. aseo_completo_de_casas\n16. cubana\n17. uber\n18. tutorias\n19. muebles\n20. gatos\n21. transporte_de_alimentos\n22. smartphone\n23. carrozas\n24. plomero\n25. taxi\n26. delivery\n27. equipos_de_sonido\n28. hamburguesas\n29. education\n30. tv\n31. motoboy\n32. musica\n33. albanil\n34. depilacion\n35. llaves_perdidas\n36. mototaxi\n37. guitar\n38. restaurant\n39. moda\n40. food\n41. manos_y_pies\n42. indriver\n43. cerrajeria_automotriz\n44. foodtruck\n45. marketing\n46. electeicidade_generale\n47. repair\n48. jardin\n49. travel_agency\n50. transport\n51. alba\n52. computadoras\n53. pies\n54. nevera\n55. musico\n56. speak_english\n57. doctor\n58. psicopedagoga\n59. electricidade\n60. bicicletas\n61. aire_aconditionado\n62. estufas\n63. educacion\n64. 

In [13]:
print(response.json()["output"]["output"])

The categories in the database are:
1. misc
2. cerrajeria
3. piano
4. cabello
5. bateria
6. french_food
7. body_treatments
8. 8_56_hierro
9. home
10. moto
11. filmmaking
12. auto
13. music
14. video
15. aseo_completo_de_casas
16. cubana
17. uber
18. tutorias
19. muebles
20. gatos
21. transporte_de_alimentos
22. smartphone
23. carrozas
24. plomero
25. taxi
26. delivery
27. equipos_de_sonido
28. hamburguesas
29. education
30. tv
31. motoboy
32. musica
33. albanil
34. depilacion
35. llaves_perdidas
36. mototaxi
37. guitar
38. restaurant
39. moda
40. food
41. manos_y_pies
42. indriver
43. cerrajeria_automotriz
44. foodtruck
45. marketing
46. electeicidade_generale
47. repair
48. jardin
49. travel_agency
50. transport
51. alba
52. computadoras
53. pies
54. nevera
55. musico
56. speak_english
57. doctor
58. psicopedagoga
59. electricidade
60. bicicletas
61. aire_aconditionado
62. estufas
63. educacion
64. planchas
65. dog_sitting
66. desarrollo_de_software
67. clases_de_ingles
68. lavado_de_

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [14]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [15]:
from langchain_core.messages import HumanMessage, AIMessage

In [16]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break
    ai = await remote_runnable.ainvoke({"input": human, "chat_history": chat_history})
    print(f"AI: {ai['output']}")
    chat_history.extend([HumanMessage(content=human),
                        AIMessage(content=ai["output"])])

AI: There are a total of 108 categories in the database.
AI: Yes, there is a category for "plomero" in the database.
AI: Bye bye human


## Stream

Please note that streaming alternates between actions and observations. It does not stream individual tokens!

To stream individual tokens, we need to use the astream events endpoint (see below).

In [19]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream(
        {"input": human, "chat_history": chat_history}
    ):
        # Agent Action
        if "actions" in chunk:
            for action in chunk["actions"]:
                print(
                    f"Calling Tool ```{action['tool']}``` with input ```{action['tool_input']}```"
                )
        # Observation
        elif "steps" in chunk:
            for step in chunk["steps"]:
                print(f"Got result: ```{step['observation']}```")
        # Final result
        elif "output" in chunk:
            print(chunk["output"])
            ai = AIMessage(content=chunk["output"])
        else:
            raise ValueError
        print("------")
    chat_history.extend([HumanMessage(content=human), ai])

AI: 
¡Hola! ¿En qué puedo ayudarte hoy?
------
AI: 
Calling Tool ```service_categories``` with input ```{}```
------
Got result: ```['misc', 'cerrajeria', 'piano', 'cabello', 'bateria', 'french_food', 'body_treatments', '8_56_hierro', 'home', 'moto', 'filmmaking', 'auto', 'music', 'video', 'aseo_completo_de_casas', 'cubana', 'uber', 'tutorias', 'muebles', 'gatos', 'transporte_de_alimentos', 'smartphone', 'carrozas', 'plomero', 'taxi', 'delivery', 'equipos_de_sonido', 'hamburguesas', '5', 'education', 'tv', 'motoboy', 'piano', 'musica', 'albanil', 'depilacion', 'llaves_perdidas', 'mototaxi', 'guitar', 'restaurant', 'moda', 'food', 'manos_y_pies', 'indriver', 'cerrajeria_automotriz', 'foodtruck', 'marketing', 'electeicidade_generale', 'repair', 'jardin', 'travel_agency', 'transport', 'piano', 'alba', 'musica', 'computadoras', 'pies', 'bateria', 'nevera', 'musico', 'speak_english', 'doctor', 'piano', 'psicopedagoga', 'electricidade', 'bicicletas', 'aire_aconditionado', 'estufas', 'musica'

In [20]:
chat_history

[HumanMessage(content='Hola'),
 AIMessage(content='¡Hola! ¿En qué puedo ayudarte hoy?'),
 HumanMessage(content='Necesito alguien que me lave el auto'),
 AIMessage(content='Puedo ayudarte a encontrar un servicio de lavado de automóviles. ¿Te gustaría que busque un proveedor de este servicio para ti?')]

## Stream Events

In [18]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break
    ai = None
    print("AI: ")
    async for event in remote_runnable.astream_events(
        {"input": human, "chat_history": chat_history},
        version="v1",
    ):
        kind = event["event"]
        if kind == "on_chain_start":
            if (
                event["name"] == "agent"
            ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
                print(
                    f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
                )
        elif kind == "on_chain_end":
            if (
                event["name"] == "agent"
            ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
                print()
                print("--")
                print(
                    f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
                )
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # Empty content in the context of OpenAI means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content
                print(content, end="|")
        elif kind == "on_tool_start":
            print("--")
            print(
                f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
            )
        elif kind == "on_tool_end":
            print(f"Done tool: {event['name']}")
            print(f"Tool output was: {event['data'].get('output')}")
            print("--")

AI: 
Starting agent: agent with input: {'input': 'Hola', 'chat_history': []}
¡|Hola|!| ¿|En| qué| puedo| ayud|arte| hoy|?|
--
Done agent: agent with output: ¡Hola! ¿En qué puedo ayudarte hoy?
AI: 
Starting agent: agent with input: {'input': 'Hay mecanicos de autos como provehedores de servicios?', 'chat_history': []}
--
Starting tool: service_categories with inputs: {}
Done tool: service_categories
Tool output was: ['misc', 'cerrajeria', 'piano', 'cabello', 'bateria', 'french_food', 'body_treatments', '8_56_hierro', 'home', 'moto', 'filmmaking', 'auto', 'music', 'video', 'aseo_completo_de_casas', 'cubana', 'uber', 'tutorias', 'muebles', 'gatos', 'transporte_de_alimentos', 'smartphone', 'carrozas', 'plomero', 'taxi', 'delivery', 'equipos_de_sonido', 'hamburguesas', '5', 'education', 'tv', 'motoboy', 'piano', 'musica', 'albanil', 'depilacion', 'llaves_perdidas', 'mototaxi', 'guitar', 'restaurant', 'moda', 'food', 'manos_y_pies', 'indriver', 'cerrajeria_automotriz', 'foodtruck', 'marketin